# Core Architecture: Interfaces

The file `swp/core/interfaces.py` defines the fundamental **Abstract Base Classes (ABCs)** for the Smart Water Platform. These interfaces are the contracts that all major components must adhere to. This ensures that the system remains modular, consistent, and 'pluggable'.

This notebook explains the main interfaces and provides simple examples of how to implement them. This is essential reading for any developer looking to extend the platform with new components.

## 1. The `Simulatable` Interface

The `Simulatable` interface is for any object whose state can be advanced over time. This is the base for all physical objects and some logical components.

**Key Methods:**
- `step(action, dt)`: Advances the simulation by one time step `dt`, given some `action`.
- `get_state()`: Returns the current state of the object.
- `set_state(state)`: Sets the object's state.
- `get_parameters()`: Returns the object's configuration parameters.

### Example Implementation

In [ ]:
from swp.core.interfaces import Simulatable, State, Parameters

class MySimulatableObject(Simulatable):
    def __init__(self):
        self._state = {'value': 0}
        self._params = {'gain': 2.0}
        
    def step(self, action: any, dt: float) -> State:
        # Example: state value increases by action * gain
        self._state['value'] += action * self._params['gain'] * dt
        return self._state

    def get_state(self) -> State:
        return self._state

    def set_state(self, state: State):
        self._state = state

    def get_parameters(self) -> Parameters:
        return self._params

# Usage
my_obj = MySimulatableObject()
print(f"Initial state: {my_obj.get_state()}")
my_obj.step(action=5, dt=1.0)
print(f"State after step: {my_obj.get_state()}")

## 2. The `Agent` Interface

The `Agent` interface is the base class for all autonomous agents in the multi-agent system. It's a simple interface with one key method.

**Key Methods:**
- `run(current_time)`: The main entry point for an agent's behavior, called periodically by the simulation harness.

### Example Implementation

In [ ]:
from swp.core.interfaces import Agent

class MyAgent(Agent):
    def __init__(self, agent_id: str):
        super().__init__(agent_id)
        self.run_count = 0
        
    def run(self, current_time: float):
        self.run_count += 1
        print(f"[{self.agent_id}] run() called at time {current_time}. Total runs: {self.run_count}")

# Usage
my_agent = MyAgent(agent_id="test_agent_007")
my_agent.run(current_time=0.0)
my_agent.run(current_time=1.0)

## 3. The `Controller` Interface

The `Controller` interface is designed to encapsulate a control algorithm. This allows the core logic (e.g., PID, MPC) to be separated from the agent that uses it.

**Key Methods:**
- `compute_control_action(observation, dt)`: Takes the current system state (`observation`) and computes the appropriate control action.

### Example Implementation

In [ ]:
from swp.core.interfaces import Controller, State
from typing import Any

class MyBangBangController(Controller):
    """A simple controller that is either fully on or fully off."""
    def __init__(self, setpoint: float):
        self.setpoint = setpoint
        
    def compute_control_action(self, observation: State, dt: float) -> Any:
        process_variable = observation.get('value', 0)
        if process_variable < self.setpoint:
            return 'ON' # Return the computed action
        else:
            return 'OFF'

# Usage
my_controller = MyBangBangController(setpoint=100.0)
current_state = {'value': 85.0}
action = my_controller.compute_control_action(current_state, 1.0)
print(f"Current value is {current_state['value']}, controller action is: {action}")

current_state = {'value': 101.0}
action = my_controller.compute_control_action(current_state, 1.0)
print(f"Current value is {current_state['value']}, controller action is: {action}")

## 4. The `PhysicalObjectInterface`

This is a specialized interface that inherits from `Simulatable` and `Identifiable`. It is the standard base class for all physical components in the `swp/simulation_identification/physical_objects/` directory. It adds a required `name` and a `set_inflow()` method used by the `SimulationHarness`.